<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/ChromaDB_Similarity_Search%20Vector_Databases_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip -q install langchain

In [3]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

In [4]:
import os
from google.colab import userdata
OPEN_AI=userdata.get('openai')
os.environ['OPENAI_API_KEY']=OPEN_AI

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [6]:
loader=DirectoryLoader('/content/new_articles/',
                       glob='./*.txt',
                       loader_cls=TextLoader)
documents=loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [8]:
texts=text_splitter.split_documents(documents)

In [11]:
persist_directory='db'
embedding=OpenAIEmbeddings()
vectordb=Chroma.from_documents(documents=texts,
                               embedding=embedding,
                               persist_directory=persist_directory)

In [14]:
#persist the db to disk
vectordb.persist()
vectordb=None

In [15]:
vectordb=Chroma(persist_directory=persist_directory,
                embedding_function=embedding)

In [16]:
#semantic/similarity search

In [21]:
retriever=vectordb.as_retriever(search_kwargs={'k':2})

In [22]:
docs=retriever.get_relevant_documents("How much money did Microsoft raise?")

In [23]:
len(docs)

2

In [24]:
retriever.search_type

'similarity'

In [27]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [29]:
qa_chain=RetrievalQA.from_chain_type(llm=OpenAI(),
                     chain_type='stuff',
                     retriever=retriever,
                     return_source_documents=True)

In [34]:
query='What is spacex?'
llm_response=qa_chain(query)

In [35]:
llm_response

{'query': 'What is spacex?',
 'result': ' SpaceX is a private aerospace company founded and led by Elon Musk, with the goal of revolutionizing space exploration and making humanity a multi-planetary species. They have developed several rockets and spacecraft, including the Falcon 9 and Dragon, and are currently working on the super-heavy launch system Starship.',
 'source_documents': [Document(page_content='SpaceX’s super-heavy launch system Starship is poised to fundamentally reshape the space economy. The 394-foot-tall vehicle, which took to the skies for the first time last month, is designed to carry a staggering amount of mass to low Earth orbit and into deep space.\n\nTechCrunch+ spoke with three pure-play space VCs — Space Capital founder and managing partner Chad Anderson, Space.VC founder and general partner Jonathan Lacoste and E2MC Ventures founder Raphael Roettgen — to learn more about how they advise founders to think through Starship’s super-heavy implications.\n\nWhile t

In [36]:
def process_llm_response(response):
  print(response['result'])
  print('\n\nSources')
  for source in response['source_documents']:
    print(source.metadata['source'])

In [37]:
process_llm_response(llm_response)


 SpaceX is a private aerospace company founded and led by Elon Musk, with the goal of revolutionizing space exploration and making humanity a multi-planetary species. They have developed several rockets and spacecraft, including the Falcon 9 and Dragon, and are currently working on the super-heavy launch system Starship.


Sources
/content/new_articles/05-07-spacex-starship-startups-future.txt
/content/new_articles/05-07-spacex-starship-startups-future.txt


In [38]:
llm=OpenAI()
llm.predict('what is yolo')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"',\n    'YOLO (You Only Look Once) is a real-time object detection algorithm. It is popular because it is fast and accurate, and can detect multiple objects in an image with a single forward pass of a neural network. YOLO works by dividing an image into a grid and predicting bounding boxes and class probabilities for each cell in the grid. It then uses non-max suppression to remove duplicate detections and outputs the final predictions. YOLO has many applications, including in self-driving cars, surveillance systems, and robotics."

In [39]:
#Pinecone